In [19]:
import pandas as pd 
import numpy as np 
import os
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)

# declaramos las credenciales al entorno de desarollo
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "..\\..\\bd\\0_docs\\datahub-deacero-adojeda.json"

# Crear date_today como el primer día del mes actual
now = datetime.now() - relativedelta(days= 3)

date_today = datetime(now.year, now.month, 1)

# Crear date_last_6m como seis meses antes de date_today
date_last_6m = date_today - relativedelta(months=6)

# Crear date_forecas como seis meses después de date_today
date_forecas = date_today + relativedelta(months=6)

# Mostrar las fechas
print("date_today:", date_today.strftime("%Y-%m-%d"))

print("date_last_6m:", date_last_6m.strftime("%Y-%m-%d"))

print("date_forecas:", date_forecas.strftime("%Y-%m-%d"))

date_today: 2025-08-01
date_last_6m: 2025-02-01
date_forecas: 2026-02-01


In [20]:
query_str = """
SELECT 
FECHA,
Y_HIST,
Y_TEST,
Y_PREDICCION,
Y_PRONOSTICOS,
NOMBRESUBDIRECCION,
NOMBREGRUPOESTADISTICO3,
VARIABLE_EXOGENA,
IMPORTANCIA,
#CORRELACIONES,
#CORR_PAIRS,
USE,
`MODEL`,
`COV%`,
`MAPE%`,
`MAPE_DEACERO%`,
WMAPE,
TEST,
FECHA_EJECUCION
FROM demanda-prj-dev.pronosticos.forecast_table
WHERE FECHA_EJECUCION = (
SELECT MAX(FECHA_EJECUCION)
FROM demanda-prj-dev.pronosticos.forecast_table
)
AND TEST = '0'
"""

client = bigquery.Client(project="demanda-prj-dev")

consulta = query_str

query = client.query(consulta)

forecast = query.to_dataframe(create_bqstorage_client=True)

forecast.sort_index(inplace=True, ascending=True)

forecast["Y_HIST"] = forecast["Y_HIST"].astype(float)

forecast["Y_PREDICCION"] = forecast["Y_PREDICCION"].astype(float)

forecast["Y_PRONOSTICOS"] = forecast["Y_PRONOSTICOS"].astype(float)

forecast["Y_TEST"] = forecast["Y_TEST"].astype(float)


c:\Users\adojeda\OneDrive - deacero.com\dev\0_envs\.bigquery\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [22]:
forecast

,FECHA,Y_HIST,Y_TEST,Y_PREDICCION,Y_PRONOSTICOS,NOMBRESUBDIRECCION,NOMBREGRUPOESTADISTICO3,VARIABLE_EXOGENA,IMPORTANCIA,USE,MODEL,COV%,MAPE%,MAPE_DEACERO%,WMAPE,TEST,FECHA_EJECUCION
0,2024-09-01,1109.791510,NaN,NaN,NaN,AGRICULTURAL DISTRIBUTION,CERCOS GRADUADAS,"['sin variable', 'Morelos_actividad_industrial...","[nan, 0.2357, 0.2148, 0.1758, 0.1722, 0.0931, ...",No Log,RandomForestRegressor,28.6723,22.1171,15.2443,15.7006,0,2025-08-28 05:49:21
1,2024-10-01,1354.432250,NaN,NaN,NaN,AGRICULTURAL DISTRIBUTION,CERCOS GRADUADAS,"['sin variable', 'Morelos_actividad_industrial...","[nan, 0.2357, 0.2148, 0.1758, 0.1722, 0.0931, ...",No Log,RandomForestRegressor,28.6723,22.1171,15.2443,15.7006,0,2025-08-28 05:49:21
2,2024-11-01,1262.169504,NaN,NaN,NaN,AGRICULTURAL DISTRIBUTION,CERCOS GRADUADAS,"['sin variable', 'Morelos_actividad_industrial...","[nan, 0.2357, 0.2148, 0.1758, 0.1722, 0.0931, ...",No Log,RandomForestRegressor,28.6723,22.1171,15.2443,15.7006,0,2025-08-28 05:49:21
3,2024-12-01,1130.115750,NaN,NaN,NaN,AGRICULTURAL DISTRIBUTION,CERCOS GRADUADAS,"['sin variable', 'Morelos_actividad_industrial...","[nan, 0.2357, 0.2148, 0.1758, 0.1722, 0.0931, ...",No Log,RandomForestRegressor,28.6723,22.1171,15.2443,15.7006,0,2025-08-28 05:49:21
4,2025-01-01,1173.528430,NaN,NaN,NaN,AGRICULTURAL DISTRIBUTION,CERCOS GRADUADAS,"['sin variable', 'Morelos_actividad_industrial...","[nan, 0.2357, 0.2148, 0.1758, 0.1722, 0.0931, ...",No Log,RandomForestRegressor,28.6723,22.1171,15.2443,15.7006,0,2025-08-28 05:49:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75307,2025-10-01,NaN,NaN,NaN,2288.754762,TURIA,VARILLA 3/8,"['sin variable', 'Yucatn_manuf', 'Serviciosdes...","[nan, 0.6179, 0.3821]",Log,GradientBoostingRegressor,17.5079,12.64,10.9453,10.5378,0,2025-08-28 05:49:21
75308,2025-11-01,NaN,NaN,NaN,2256.695767,TURIA,VARILLA 3/8,"['sin variable', 'Yucatn_manuf', 'Serviciosdes...","[nan, 0.6179, 0.3821]",Log,GradientBoostingRegressor,17.5079,12.64,10.9453,10.5378,0,2025-08-28 05:49:21
75309,2025-12-01,NaN,NaN,NaN,2385.062985,TURIA,VARILLA 3/8,"['sin variable', 'Yucatn_manuf', 'Serviciosdes...","[nan, 0.6179, 0.3821]",Log,GradientBoostingRegressor,17.5079,12.64,10.9453,10.5378,0,2025-08-28 05:49:21
75310,2026-01-01,NaN,NaN,NaN,2054.963760,TURIA,VARILLA 3/8,"['sin variable', 'Yucatn_manuf', 'Serviciosdes...","[nan, 0.6179, 0.3821]",Log,GradientBoostingRegressor,17.5079,12.64,10.9453,10.5378,0,2025-08-28 05:49:21
